# An Improved Reliability-Based Approach to Sepcifying Fire Resistance Periods For Buildings in England

Analysis documentation for the research topic submitted for ASFE 2021.

_2021 April, Ian F._

In the design of fire safety provisions for straightforward/common buildings in England the appropriate structural fire resistance is selected from guidance based upon building height and occupancy characteristics. This has been expanded upon in contemporary guidance to include consideration of ventilation conditions informed by work carried out in 2004 by Kirby, et al. in their paper “A new approach to specifying fire resistance periods” and has been implemented in British Standard, BS 9999:2017. In the work of Kirby, et al., a probabilistic time equivalence analysis was carried out using Monte Carlo simulations (MCS). Stochastic parameters were used to produce a range of credible design fires. These were primarily fuel load density, compartment geometry and ventilation opening size. The design fires were generated using the parametric fire model in EN 1991-1-2 and the contribution of sprinklers was considered through a reduced fire load. To link to existing fire resistance recommendations which did not consider ventilation, the safety/reliability target was calibrated to align with statutory guidance, Approved Document B (ADB), using a medium-rise office as an anchor point.

This study revisits the work of Kirby et al., resolving key limitations and incorporating advancements in the field to present a new approach to assessing the recommended fire resistance for structures and proposes a revised fire resistance design table for England. To seek alignment with general structural design principles/requirements, as defined in Approved Document A (ADA), safety targets are expressed in function of consequence classes in lieu of building height and use. Other key advancements include improved stochastic parameters, the use of travelling fires where post-flashover parametric fires are unrealistic (i.e., a more explicit consideration of large enclosure fire dynamics), and consideration of sprinkler contribution in the event tree in terms of their ability to mitigate structurally significant fire occurrences.

The improved approach presented provides a more up to date method for defining appropriate fire resistance for straightforward/common buildings in England.

In [ ]:
# Dependencies

import numpy as np
import pandas as pd
import pprint as pp
import matplotlib.pyplot as plt
from os import path
from scipy import stats
from sys import platform
from typing import Callable
from tqdm.notebook import tqdm
from scipy.interpolate import interp1d
from IPython.display import display, HTML

from sfeprapy.mcs2 import MCS2
from sfeprapy.mcs.mcs_gen_2 import InputParser

plt.style.use('seaborn-white')

# User defined, project folder containing inputs
if platform == "darwin":
    fp_project = r'/Volumes/ssd01/projects_fse/ASFE2021/01-analysis'
    path.exists(fp_project)
elif platform == "win32":
    fp_project = r'D:\projects_fse\!ASFE2021\01-analysis'
    path.exists(fp_project)

# Helper function
def get_fr_func(teq: np.ndarray) -> Callable:
    hist, edges = np.histogram(teq, bins=np.arange(0, max(361, max(teq[teq<np.inf])+0.5), 0.5))
    x, y = (edges[:-1] + edges[1:]) / 2, np.cumsum(hist / np.sum(hist))
    func_fr = interp1d(y, x, bounds_error=False)
    return func_fr

# Helper function
def get_teq_fractile_func(teq: np.ndarray) -> Callable:
    hist, edges = np.histogram(teq, bins=np.arange(0, max(361, max(teq[teq<np.inf])+0.5), 0.5))
    x, y = (edges[:-1] + edges[1:]) / 2, np.cumsum(hist / np.sum(hist))
    func_teq = interp1d(x, y, bounds_error=False)
    return func_teq

# Helper function
def get_teq_fractile(teq: np.ndarray, fr: list = list(range(30, 181, 15))) -> np.ndarray:
    return get_teq_fractile_func(teq = teq)(fr)

# Helper function
def get_teq_fractile_dict(teq_dict: dict, fr: list = list(range(30, 181, 15)), is_print: bool = True) -> dict:
    """Prints CFD of given time equivalence values"""
    res_dict = dict(Time=fr)
    for case, teq in teq_dict.items():
        res_dict[case] = get_teq_fractile(teq=teq, fr=fr)
    if is_print is True:
        print(pd.DataFrame.from_dict(res_dict))
    return res_dict

# Helper function
def print_df(df: pd.DataFrame):
    display(HTML(df.to_html()))

## Benchmark against Kirby et al

### Run time equivalence Monte Carlo simulations

The analysis is comprised of three cases: Office, Resiential and Retail. All input parameters are inline with Kirby et al.

#### Inputs

In [ ]:
fn = '0-trial_00.xlsx'

if platform == 'win32':
    fp_inputs = path.realpath(path.join(r'D:\projects_fse\ASFE2021\01-analysis', fn[2:10], fn))
elif platform == 'darwin':
    fp_inputs = path.realpath(path.join(r'/Volumes/ssd01/projects_fse/ASFE2021/01-analysis', fn[2:10], fn))

df_inputs = pd.read_excel(fp_inputs, index_col='case_name')
df_inputs.loc[[
    'fire_load_density:dist', 'fire_load_density:mean', 'fire_load_density:sd', 
    'fire_hrr_density:dist', 'fire_hrr_density:lbound', 'fire_hrr_density:ubound', 
    'room_height:dist', 'room_height:lbound', 'room_height:ubound', 
    'room_floor_area:dist', 'room_floor_area:lbound', 'room_floor_area:ubound', 
    'room_breadth_depth_ratio:dist', 'room_breadth_depth_ratio:lbound', 'room_breadth_depth_ratio:ubound', 
    'window_height_room_height_ratio:dist', 'window_height_room_height_ratio:lbound', 'window_height_room_height_ratio:ubound', 
    'window_area_floor_ratio:dist', 'window_area_floor_ratio:lbound', 'window_area_floor_ratio:ubound'
]]

#### Simulation

In [ ]:
# Run MCS
cases_to_run=['Office', 'Residential', 'Retail']

mcs_1 = MCS2(print_stats=True)
mcs_1.inputs = fp_inputs
mcs_1.n_threads = 6
mcs_1.run_mcs(cases_to_run=cases_to_run)

#### Results

Inspect simulation results to ensure the simulations are complete successfully.

In [ ]:
# Inspect simulation results
mcs_out_1 = mcs_1.mcs_out
dict_teq = {case: mcs_out_1.loc[mcs_out_1['case_name'] == case]["solver_time_equivalence_solved"] / 60.0 for case in cases_to_run}
dict_teq_fractile = get_teq_fractile_dict(dict_teq, is_print=False, fr=np.linspace(0, 180, 100))

fig, ax = plt.subplots(figsize=(2.5,2.5), dpi=100)
for case in cases_to_run:
    ax.plot(dict_teq_fractile['Time'], dict_teq_fractile[case], label=case)
ax.legend(fancybox=False, frameon=True, loc=0, fontsize='x-small', title_fontsize='x-small').set_visible(True)
ax.set_xlabel('Time [$min$]', fontsize='small')
ax.set_ylabel('CFD', fontsize='small')
ax.set_xlim(30, 180)
ax.set_ylim(0, 1)
ax.set_xticks(np.arange(30, 181, 15,))
ax.set_yticks(np.arange(0, 1.1, 0.1))
ax.tick_params(labelsize='x-small')
ax.grid(True, ls='--')

plt.tight_layout()
fig.savefig('teq_1.png', dpi=300)

It is obvious Residential produced much worse time equivalence results comparing to the other two, primarily due to higher fuel load density.

#### Comparing agsint Kirby et al

| FR  | Office | Retail  | Residential |
|-----|--------|---------|-------------|
| 30  | 0.464  | 0.40416 | 0.16901     |
| 45  | 0.6474 | -       | 0.19285     |
| 60  | 0.8    | 0.73384 | 0.36078     |
| 75  | 0.8963 | -       | 0.62947     |
| 90  | 0.928  | 0.91238 | 0.84399     |
| 105 | 0.9727 | -       | 0.95341     |
| 120 | 0.982  | 0.96824 | 0.99133     |

In [ ]:
data = {
    'Office': {
        'Equivalent of time exposure': [30, 60, 90, 120, 150],
        'CDF (Kirby et al)': [0.464, 0.8, 0.928, 0.982, 0.996],
        'CDF (SFEPRAPY)': get_teq_fractile(
            teq=mcs_out_1.loc[mcs_out_1['case_name'] == 'Office']["solver_time_equivalence_solved"] / 60.0,
            fr=[30, 60, 90, 120, 150]
        )
    },
    'Retail': {
        'Equivalent of time exposure': [30, 60, 90, 120],
        'CDF (Kirby et al)': [0.4041621029572837, 0.7338444687842278, 0.9123767798466594, 0.968236582694414],
        'CDF (SFEPRAPY)': get_teq_fractile(
            teq=mcs_out_1.loc[mcs_out_1['case_name'] == 'Retail']["solver_time_equivalence_solved"] / 60.0,
            fr=[30, 60, 90, 120]
        )
    },
    'Dwelling': {
        'Equivalent of time exposure': [60, 75, 90, 105, 120],
        'CDF (Kirby et al)': [0.36078, 0.62947, 0.84399, 0.95341, 0.99133,],
        'CDF (SFEPRAPY)': get_teq_fractile(
            teq=mcs_out_1.loc[mcs_out_1['case_name'] == 'Residential']["solver_time_equivalence_solved"] / 60.0,
            fr=[60, 75, 90, 105, 120]
        )
    }
}

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(2.25*3, 2.5), sharex=True, sharey=True, dpi=100)

for i, k in enumerate(data.keys()):
    ax, v = axes[i], data[k]
    ax.scatter(v['Equivalent of time exposure'], v['CDF (Kirby et al)'], label='CDF (Kirby et al)', marker='o', facecolors='none', edgecolors='k')
    ax.scatter(v['Equivalent of time exposure'], v['CDF (SFEPRAPY)'], label='CDF (SFEPRAPY)', marker='^', facecolors='none', edgecolors='k')
    ax.legend(title=k, fancybox=False, frameon=True, loc=0, fontsize='x-small', title_fontsize='x-small').set_visible(True)
    ax.set_xlabel('Equivalent of time exposure [$min$]', fontsize='small')
    ax.set_xlim(15, 135)
    ax.set_xticks(np.arange(30, 121, 15))
    ax.grid(which='both', ls='--')
    ax.tick_params(labelsize='x-small')

ax = axes[0]
ax.set_ylabel('CFD', fontsize='small')
ax.set_ylim(0, 1)
ax.set_yticks(np.arange(0,1.1,0.1));

plt.tight_layout()
fig.savefig('benchmarking_against_kirby_1.png', dpi=300)

## SFEPRAPY including travelling fire

In [ ]:
# Update inputs to include travelling fire model
df_inputs_2 = df_inputs.copy()
df_inputs_2.loc['fire_mode'] = 3
df_inputs_2.loc['n_simulations'] = 100000

# Run MCS
cases_to_run=['Office', 'Residential', 'Retail']
mcs_2 = MCS2(print_stats=False)
mcs_2.inputs = df_inputs_2
mcs_2.n_threads = 10
mcs_2.run_mcs(cases_to_run=cases_to_run)

In [ ]:
# Inspect simulation results
mcs_out_2 = mcs_2.mcs_out
dict_teq = {case: mcs_out_2.loc[mcs_out_2['case_name'] == case]["solver_time_equivalence_solved"] / 60.0 for case in cases_to_run}
dict_teq_fractile = get_teq_fractile_dict(dict_teq, is_print=False, fr=np.linspace(0, 180, 100))

fig, ax = plt.subplots(figsize=(3.5,3.5), dpi=100)
for case in cases_to_run:
    ax.plot(dict_teq_fractile['Time'], dict_teq_fractile[case], label=case)
ax.legend().set_visible(True)
ax.set_xlabel('Time [$min$]')
ax.set_ylabel('CFD')
ax.set_xlim(30, 180)
ax.set_ylim(0, 1)
ax.set_xticks(np.arange(30, 181, 15,))
ax.set_yticks(np.arange(0, 1.1, 0.1))
ax.grid(True, ls='--');

In [ ]:
# Compare against Kirby

data = {
    'Office': {
        'Equivalent of time exposure': [30, 60, 90, 120, 150],
        'CDF (Kirby et al)': [0.464, 0.8, 0.928, 0.982, 0.996],
        'CDF (SFEPRAPY)': get_teq_fractile(
            teq=mcs_out_2.loc[mcs_out_2['case_name'] == 'Office']["solver_time_equivalence_solved"] / 60.0,
            fr=[30, 60, 90, 120, 150]
        )
    },
    'Retail': {
        'Equivalent of time exposure': [30, 60, 90, 120],
        'CDF (Kirby et al)': [0.4041621029572837, 0.7338444687842278, 0.9123767798466594, 0.968236582694414],
        'CDF (SFEPRAPY)': get_teq_fractile(
            teq=mcs_out_2.loc[mcs_out_2['case_name'] == 'Retail']["solver_time_equivalence_solved"] / 60.0,
            fr=[30, 60, 90, 120]
        )
    },
    'Dwelling': {
        'Equivalent of time exposure': [60, 75, 90, 105, 120],
        'CDF (Kirby et al)': [0.36078, 0.62947, 0.84399, 0.95341, 0.99133,],
        'CDF (SFEPRAPY)': get_teq_fractile(
            teq=mcs_out_2.loc[mcs_out_2['case_name'] == 'Residential']["solver_time_equivalence_solved"] / 60.0,
            fr=[60, 75, 90, 105, 120]
        )
    }
}

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(2.25*3, 2.5), sharex=True, sharey=True, dpi=100)

for i, k in enumerate(data.keys()):
    ax, v = axes[i], data[k]
    ax.scatter(v['Equivalent of time exposure'], v['CDF (Kirby et al)'], label='CDF (Kirby et al)', marker='o', facecolors='none', edgecolors='k')
    ax.scatter(v['Equivalent of time exposure'], v['CDF (SFEPRAPY)'], label='CDF (SFEPRAPY)', marker='^', facecolors='none', edgecolors='k')
    ax.legend(title=k, fancybox=False, frameon=True, loc=0, fontsize='x-small', title_fontsize='x-small').set_visible(True)
    ax.set_xlabel('Equivalent of time exposure [$min$]', fontsize='small')
    ax.set_xlim(15, 135)
    ax.set_xticks(np.arange(30, 121, 15))
    ax.grid(which='both', ls='--')
    ax.tick_params(labelsize='x-small')

ax = axes[0]
ax.set_ylabel('CFD', fontsize='small')
ax.set_ylim(0, 1)
ax.set_yticks(np.arange(0,1.1,0.1))

plt.tight_layout()
fig.savefig('benchmarking_against_kirby_2.png', dpi=300)

## Failure Probability

In [ ]:
failure_parameters = {
    "Residential": dict(p1=6.5e-7, p2=0.2, p3=0.25, p4=1),
    "Office": dict(p1=3.0e-7, p2=0.2, p3=0.25, p4=1),
    "Retail": dict(p1=4.0e-7, p2=0.2, p3=0.25, p4=1),
}

In [ ]:
def P_f_fi_func(p1: float, p2: float, p3: float, p4: float, A: np.ndarray, teq: np.ndarray) -> pd.DataFrame:
    P_f_fi_ = dict(FR = np.arange(30, 181, 1))
    p5 = 1 - get_teq_fractile(teq=teq, fr=P_f_fi_['FR'])
    for i in A:
        P_f_fi_[f'{i:.0f} m²'] = p1 * i * p2 * p3 * p4 * p5
    return pd.DataFrame.from_dict(P_f_fi_).set_index('FR')

In [ ]:
As = np.arange(2000, 10001, 2000)
As = [1000, 2000, 4000, 8000, 16000, 32000]
df_residential = P_f_fi_func(
    p1=6.5e-7,
    p2=0.2,
    p3=0.25,
    p4=1,
    A=As,
    teq=mcs_out_2.loc[mcs_out_2['case_name'] == 'Residential']["solver_time_equivalence_solved"] / 60.0,
)
df_office = P_f_fi_func(
    p1=6.5e-7,
    p2=0.2,
    p3=0.25,
    p4=1,
    A=As,
    teq=mcs_out_2.loc[mcs_out_2['case_name'] == 'Office']["solver_time_equivalence_solved"] / 60.0,
)
df_retail = P_f_fi_func(
    p1=6.5e-7,
    p2=0.2,
    p3=0.25,
    p4=1,
    A=As,
    teq=mcs_out_2.loc[mcs_out_2['case_name'] == 'Retail']["solver_time_equivalence_solved"] / 60.0,
)
# display(HTML(f'<b>Residential</b>'))
# print_df(df_residential)
# df_residential.plot()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(2.3*3, 2.5), sharex=True, sharey=True, dpi=100)

dfs = dict(Residential=df_residential, Office=df_office, Retail=df_retail)
i = 0
for i, key in enumerate(dfs):
    ax = axes[i]
    df = dfs[key]
    x = df.index.values
    ys = df.to_dict(orient='list')
    for j, y in ys.items():
        ax.plot(x, y, label=j)
    ax.legend(title=key, fancybox=False, frameon=True, loc=0, title_fontsize='x-small', fontsize='xx-small', ncol=1).set_visible(True)
    ax.set_yscale('log')
    ax.set_xlabel('Equivalent of time exposure [$min$]', fontsize='small')
    ax.set_xlim(30, 180)
    ax.set_xticks(np.arange(30, 181, 15))
    ax.grid(which='both', ls='--')
    ax.tick_params(labelsize='x-small')
    i += 1

ax = axes[0]
ax.set_ylabel(r'$P_{f,fi}$ [${year}^{-1}$]', fontsize='small');
ax.set_ylim(1e-7, 1e-3);
# ax.set_yticks(np.arange(0,1.1,0.1));

plt.tight_layout()
fig.savefig('failure_probabilities_against_area.png', dpi=300)

In [ ]:
dict_P_a_fi = {
    'Building height [m]': ['0-5', '5-11', '11-18', '18-30', '30-50', '>50'],
    'Residential': [3.7, 3.7, 4.2, 4.2, 4.2, 4.4],
    'Office': [3.7, 3.7, 4.2, 4.2, 4.2, 4.4],
    'Retail': [3.7, 3.7, 4.2, 4.2, 4.2, 4.4],
}

n = stats.norm(loc=0, scale=1)
for case in cases_to_run:
    dict_P_a_fi[case] = [n.cdf(-i) for i in dict_P_a_fi[case]]

df_P_a_fi = pd.DataFrame.from_dict(dict_P_a_fi).set_index('Building height [m]').transpose()

print_df(df_P_a_fi)

In [ ]:
x = [-np.inf, 5, 18, 50, np.inf]
y = [3.7, 3.7, 4.2, 4.4, 4.4]
func_beta = interp1d(x, y, bounds_error=False)

fig, ax = plt.subplots(figsize=(3, 2.5), dpi=100)
ax.plot(np.linspace(0, 100, 1000), func_beta(np.linspace(0, 100, 1000)))
ax.set_xticks([5, 18, 50])
ax.set_xticklabels(['≤5 m', '18 m', '≥50 m'])
ax.set_xlim(0, 60)
ax.set_xlabel('Building height [m]', fontsize='small')
ax.set_yticks([3.7, 4.2, 4.4])
ax.set_ylabel(r'$\beta$', fontsize='small')
ax.grid(which='both', ls='--')
ax.tick_params(labelsize='x-small')

sec_ax_y = ax.secondary_yaxis('right', functions=(lambda x: n.cdf(-x), lambda x: -n.ppf(x)))
sec_ax_y.set_yticks([n.cdf(-3.7), n.cdf(-4.2), n.cdf(-4.4)])
sec_ax_y.ticklabel_format(style='sci', scilimits=(0, 0))
sec_ax_y.set_ylabel(r'$P_{a,fi}$ [${year}^{-1}$]', fontsize='small')
sec_ax_y.tick_params(labelsize='x-small')

plt.tight_layout()
fig.savefig('building_height_vs_beta_failure.png', dpi=300)

In [ ]:
# helper function
def func_fr_from_area_height(
        area: np.ndarray,
        height: np.ndarray,
        height_vs_beta: list,
        teq: np.ndarray, p_1: float, p_2: float, p_3: float, p_4: float,
):
    '''
    Solve:
    P_f_fi = P_a_fi
    p_1 * A * p_2 * p_3 * p_4 * p_5 = P_a_fi(H)
    p_5 = P_a_fi(H) / (p_1 * A * p_2 * p_3 * p_4)
    1 - teq_cdf = P_a_fi(H) / (p_1 * A * p_2 * p_3 * p_4)
    teq_cdf = 1 - P_a_fi(H) / (p_1 * A * p_2 * p_3 * p_4)
    '''
    if isinstance(area, np.ndarray) or isinstance(height, np.ndarray):
        assert isinstance(area, np.ndarray) 
        assert isinstance(height, np.ndarray)
        assert area.shape == height.shape
        area.astype(np.float64)
        height.astype(np.float64)
    
    # work out P_a_fi = f(height)
    func_P_a_fi_from_height = interp1d(height_vs_beta[0], [stats.norm(loc=0, scale=1).cdf(-i) for i in height_vs_beta[1]], bounds_error=False)

    # work out teq = f(fractile)
    hist, edges = np.histogram(teq, bins=np.arange(0, max(361, max(teq[teq < np.inf]) + 0.5), 0.5))
    teq_i, cdf_i = (edges[:-1] + edges[1:]) / 2, np.cumsum(hist / np.sum(hist))
    func_teq_from_fractile = interp1d(cdf_i, teq_i, bounds_error=False, fill_value=(0., 9999.))
    
    # solve/interpolation
    P_a_fi = func_P_a_fi_from_height(height)
    teq_cdf = 1 - np.divide(P_a_fi, p_1 * area * p_2 * p_3 * p_4, dtype=np.float64)
    teq = func_teq_from_fractile(teq_cdf)
    
    return teq

In [ ]:
# calcs
xx, yy = np.meshgrid(
    np.linspace(0, 50, 100), 
    np.linspace(2000, 50000, 1000),
)

zz = func_fr_from_area_height2(
    area=yy,
    height=xx,
    height_vs_beta=[
        [-np.inf, 5, 18, 50, np.inf], 
        [3.7, 3.7, 4.2, 4.4, 4.4]
    ],
    teq=mcs_out_2[mcs_out_2['case_name'] == 'Residential']["solver_time_equivalence_solved"] / 60.0,
    **dict(p_1=6.5e-7,p_2=0.2,p_3=0.0625,p_4=.09,)
)

# visualisation
fig, ax = plt.subplots(figsize=(3.5, 3.5), dpi=100)

levels = [-1, 60, 90, 120, 150, 180]
cs = ax.contour(xx, yy, zz, levels=levels, linewidths=0.5, colors='k', antialiased=True)
cf = ax.contourf(xx, yy, zz, levels=levels, cmap='coolwarm', alpha=0.6)

ax.clabel(cs, cs.levels, inline=True, fmt=lambda x: f'{x:.0f}', fontsize='small')

ax.set_xlabel('Building height [$m$]', fontsize='small')
ax.set_ylabel('Total floor area [$m^2$]', fontsize='small')
ax.set_xticks([5, 10, 15, 18, 20, 25, 30, 35, 40, 45, 50])
ax.set_xlim(5, 50)

ax.tick_params(labelsize='x-small')
ax.ticklabel_format(style='sci', scilimits=(0, 4), axis='y', useMathText=True)
ax.yaxis.offsetText.set_fontsize('x-small')
ax.grid(which='both', ls='--')

plt.show()

In [ ]:
# calcs
xx, yy = np.meshgrid(
    np.linspace(0, 50, 100), 
    np.linspace(50, 50000, 1000),
)

zz = func_fr_from_area_height2(
    area=yy,
    height=xx,
    height_vs_beta=[
        [-np.inf, 5, 18, 50, np.inf], 
        [3.7, 3.7, 4.2, 4.4, 4.4]
    ],
    teq=mcs_out_2[mcs_out_2['case_name'] == 'Office']["solver_time_equivalence_solved"] / 60.0,
    **dict(p_1=3e-7,p_2=0.2,p_3=0.2,p_4=.07,)
)

# visualisation
fig, ax = plt.subplots(figsize=(3.5, 3.5), dpi=100)

levels = [-1, 60, 75, 90, 120, 150, 180]
cs = ax.contour(xx, yy, zz, levels=levels, linewidths=0.5, colors='k', antialiased=True)
cf = ax.contourf(xx, yy, zz, levels=levels, cmap='coolwarm', alpha=0.6)

ax.clabel(cs, cs.levels, inline=True, fmt=lambda x: f'{x:.0f}', fontsize='small')

ax.set_xlabel('Building height [$m$]', fontsize='small')
ax.set_ylabel('Total floor area [$m^2$]', fontsize='small')
ax.set_xticks([5, 10, 15, 18, 20, 25, 30, 35, 40, 45, 50])
ax.set_xlim(5, 50)

ax.tick_params(labelsize='x-small')
ax.ticklabel_format(style='sci', scilimits=(0, 4), axis='y', useMathText=True)
ax.yaxis.offsetText.set_fontsize('x-small')
ax.grid(which='both', ls='--')

plt.show()

In [ ]:
# calcs
xx, yy = np.meshgrid(
    np.linspace(0, 50, 100), 
    np.linspace(2000, 500000, 1000),
)

zz = func_fr_from_area_height2(
    area=yy,
    height=xx,
    height_vs_beta=[
        [-np.inf, 5, 18, 50, np.inf], 
        [3.7, 3.7, 4.2, 4.4, 4.4]
    ],
    teq=mcs_out_2[mcs_out_2['case_name'] == 'Retail']["solver_time_equivalence_solved"] / 60.0,
    **dict(p_1=4e-7,p_2=0.2,p_3=0.2,p_4=1,)
)

# visualisation
fig, ax = plt.subplots(figsize=(3.5, 3.5), dpi=100)

levels = [-1, 60, 90, 120, 150, 180]
cs = ax.contour(xx, yy, zz, levels=levels, linewidths=0.5, colors='k', antialiased=True)
cf = ax.contourf(xx, yy, zz, levels=levels, cmap='coolwarm', alpha=0.6)

ax.clabel(cs, cs.levels, inline=True, fmt=lambda x: f'{x:.0f}', fontsize='small')

ax.set_xlabel('Building height [$m$]', fontsize='small')
ax.set_ylabel('Total floor area [$m^2$]', fontsize='small')
ax.set_xticks([5, 10, 15, 18, 20, 25, 30, 35, 40, 45, 50])
ax.set_xlim(5, 50)

ax.tick_params(labelsize='x-small')
ax.ticklabel_format(style='sci', scilimits=(0, 4), axis='y', useMathText=True)
ax.yaxis.offsetText.set_fontsize('x-small')
ax.grid(which='both', ls='--')

plt.show()